<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [10]</a>'.</span>

In [1]:
meas = "RNA"
layer = "raw"
census = "2024-07-01"
tissue_general = "tongue"
var_filter = None
dask_chunk_size = 0  # Non-Dask mode, by default

In [2]:
# Parameters
tissue_general = "tongue"
dask_chunk_size = "10k_000"


In [3]:
from typing import Literal
import anndata as ad
import scanpy as sc
from somacore import AxisQuery
from tiledbsoma import Experiment, SOMATileDBContext

DEFAULT_CONFIG = {
    "vfs.s3.no_sign_request": "true",
    "vfs.s3.region": "us-west-2"
}
CENSUS_S3 = "s3://cellxgene-census-public-us-west-2/cell-census"

species = "homo_sapiens"
soma_uri = f"{CENSUS_S3}/{census}/soma"
exp_uri = f"{soma_uri}/census_data/{species}"

exp = Experiment.open(
    exp_uri,
    context=SOMATileDBContext(tiledb_config=DEFAULT_CONFIG)
)
exp

/home/ubuntu/c/scverse/anndata/src/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/ubuntu/c/scverse/anndata/src/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/ubuntu/c/scverse/anndata/src/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/home/ubuntu/c/scverse/anndata/src/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/home/ubuntu/c/scverse/anndata/src/anndata/utils.py:429: FutureWarning: Importing read_mtx from `anndata` is deprecated. Import anndata.io.read_mtx instead.
  warnings.warn(msg, FutureWarning)
/home/ubuntu/c/scverse/anndat

<Experiment 's3://cellxgene-census-public-us-west-2/cell-census/2024-07-01/soma/census_data/homo_sapiens' (open for 'r') (2 items)
    'ms': 's3://cellxgene-census-public-us-west-2/cell-census/2024-07-01/soma/census_data/homo_sapiens/ms' (unopened)
    'obs': 's3://cellxgene-census-public-us-west-2/cell-census/2024-07-01/soma/census_data/homo_sapiens/obs' (unopened)>

In [4]:
%%time
obs_filter = f'tissue_general == "{tissue_general}"' if tissue_general else None
query = exp.axis_query(
    measurement_name=meas,
    obs_query=AxisQuery(value_filter=obs_filter) if obs_filter else None,
    var_query=AxisQuery(value_filter=var_filter) if var_filter else None,
)

CPU times: user 4.83 ms, sys: 2.47 ms, total: 7.3 ms
Wall time: 101 ms


In [5]:
%%time
query.n_obs

CPU times: user 445 ms, sys: 240 ms, total: 685 ms
Wall time: 401 ms


45060

In [6]:
from dask.distributed import Client, LocalCluster
if dask_chunk_size:
    cluster = LocalCluster()
    client = Client(cluster)
else:
    dask_chunk_size = None
    client = None
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 61.45 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45061,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 61.45 GiB
Comm: tcp://127.0.0.1:44689,Total threads: 4
Dashboard: http://127.0.0.1:36321/status,Memory: 15.36 GiB
Nanny: tcp://127.0.0.1:42329,


## HVG

In [7]:
%%time
add = query.to_anndata(layer, dask_chunk_size=dask_chunk_size)
add

TypeError: ExperimentAxisQuery.to_anndata() got an unexpected keyword argument 'dask_chunk_size'

In [8]:
%%time
sc.pp.normalize_total(add)

NameError: name 'add' is not defined

In [9]:
%%time
sc.pp.log1p(add)

NameError: name 'add' is not defined

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [10]:
%%time
hvg = sc.pp.highly_variable_genes(add, inplace=False, subset=True)
hvg

NameError: name 'add' is not defined

---
Census `tissue_general` counts, for reference:
```python
exp_obs = exp.obs.read().concat().to_pandas()
exp_obs.tissue_general.value_counts()
```
```
tissue_general
brain                       26281059
blood                       10835244
lung                         6231233
breast                       5555979
eye                          4190842
heart                        3629952
kidney                       2083054
liver                        1815408
small intestine              1237182
skin of body                 1045024
endocrine gland               979667
respiratory system            944355
bone marrow                   813373
spleen                        751041
lymph node                    717899
colon                         714413
placenta                      638786
reproductive system           570488
adrenal gland                 560114
nose                          470445
adipose tissue                468603
stomach                       446348
prostate gland                348664
fallopian tube                250178
pancreas                      250161
esophagus                     215951
digestive system              211912
musculature                   189452
large intestine               180996
pleural fluid                 179134
yolk sac                      169725
embryo                        165937
uterus                        148198
mucosa                        131978
spinal cord                   117463
exocrine gland                115722
intestine                     104490
immune system                  89046
bladder organ                  82797
vasculature                    63667
ovary                          53751
lamina propria                 45230
tongue                         45060
central nervous system         31780
esophagogastric junction       29105
axilla                         19792
pleura                         19695
skeletal system                14680
saliva                         14502
omentum                        14003
testis                         13211
tendon of semitendinosus       10533
gallbladder                     9769
scalp                           3029
ureter                          2390
Name: count, dtype: int64
```